In [2]:
from utils import keys_to_counts, get_keys

In [3]:
import pandas as pd

data_path = "D:\\studium\\analyse\\data_prep\\src\\_2_preproc_pipeline_en\\data_preprocessed\\2_tokenize_us.csv"
df = pd.read_csv(data_path)

In [4]:
df.head()

,id,pub_year,text_preproc2
0,US020240139944A1,2024,arm operating system include control device pl...
1,US020240139768A1,2024,coating device corresponding coating process d...
2,US020240139853A1,2024,tip separation apparatus welding gun disclose ...
3,US020240139933A1,2024,device method controlled motion tool present i...
4,US020240139934A1,2024,teaching method program store medium execute t...


In [5]:
# vectorize the text data with tf-idf
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=20, max_df=0.9)

x_vec = vectorizer.fit_transform(df['text_preproc2'])

In [6]:
vectorizer.get_feature_names_out().__len__()

1736

In [7]:
n_topics = 8

In [8]:
# Define Search Param
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV


search_params = {
    'n_components': [n_topics],
    'learning_decay': [0.6]
}
# Init the Model
lda = LatentDirichletAllocation(max_iter=5, learning_method='online', learning_offset=50., random_state=0)
# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)
# Do the Grid Search
model.fit(x_vec)

GridSearchCV(estimator=LatentDirichletAllocation(learning_method='online',
                                                 learning_offset=50.0,
                                                 max_iter=5, random_state=0),
             param_grid={'learning_decay': [0.6], 'n_components': [8]})

In [9]:
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(x_vec))

Best Model's Params:  {'learning_decay': 0.6, 'n_components': 8}
Best Log Likelihood Score:  -692924.5308594698
Model Perplexity:  465.4696900440025


#### 1
```
Best Model's Params:  {'learning_decay': 0.7, 'n_components': 4}
Best Log Likelihood Score:  -744378.7146935398
Model Perplexity:  579.4436712246056
```
#### 2
```
Best Model's Params:  {'learning_decay': 0.6, 'n_components': 8}
Best Log Likelihood Score:  -692924.5308594698
Model Perplexity:  465.4696900440025

vectorizer = CountVectorizer(min_df=20, max_df=0.9)
```



In [10]:
lda_output = best_lda_model.transform(x_vec)

In [11]:
import joblib
from pathlib import Path

us_outpath = Path("../output/us")
model_outpath = us_outpath / "lda_model.pkl"
ldaout_outpath = us_outpath / "lda_output.pkl"
vec_outpath = us_outpath / "vectorizer.pkl"
# Save the LDA model
joblib.dump(best_lda_model, model_outpath) 
joblib.dump(vectorizer, vec_outpath)
joblib.dump(lda_output, ldaout_outpath); 

In [ ]:
#this is by far the easiest way to load a pre-trained sklearn lda model onto pyLDAvis
#there are other means of using this visible on pyLDAvis' main repo

# import pyLDAvis
# import pyLDAvis.sklearn
# pyLDAvis.enable_notebook()

# feature_names = vectorizer.get_feature_names_out()

# vectorizer.get_feature_names = lambda: feature_names

# display_data = pyLDAvis.sklearn.prepare(best_lda_model, #our pre-trained LDA model
#                          x_vec, #this gives us our document-term matrix
#                         vectorizer) #the vectoriser object
                        
# pyLDAvis.display(display_data)  

In [ ]:
import pandas as pd

orig_data_path = "D:\\studium\\analyse\\data_prep\\src\\_2_preproc_pipeline_en\\data_preprocessed\\1_txt_cleaning_us.csv"
df_most_representative_docs = m_utils.get_most_reprdocs(df['id'], lda_output, orig_data_path, best_lda_model, 5)

In [ ]:
most_reprdoc_outpath = "../output/us/most_repr_docsptopic.csv"
df_most_representative_docs.to_csv(most_reprdoc_outpath, index=False)